# S2W5D5: 编码器整体 - 智慧的摩天大楼

## 1 理论：什么是 Context Vector？

在写代码之前，必须先理解：我们费这么大劲，输入一堆数字，最后输出的这一堆数字到底是什么？

- **输入**：一串孤立的ID：
    - 例如：`[“苹果”, “掉”, “在”, “地上”]`。
    - 这时候的“苹果”只是字典里的一个词，跟“牛顿”或者“手机”没关系。
- **经过N层Enocder之后**：
    - 每一层的Attention都在让词与词发生“化学反应”。
    - 第1层：知道“苹果”和“掉”有关系。
    - 第6层：知道这里的“苹果”是指水果（因为它掉在地上），而不是手机。
- **输出（Context Vectors）**：一串**融合了上下文**的向量。
    - 此时的$Vector_{\text{苹果}}$已经不再是单纯的“苹果”了，它包含了“掉落”、“物理实体”等丰富的信息。
    - **具身视角**: 机器人拿到这个 Vector，就知道该去**抓取**（对待物体），而不是去**充电**（对待电子产品）。

## 2 代码实现：堆叠艺术

我们将实现 `TransformerEncoder` 类。

**核心组件**:

1. **Embeddings**: 把 ID 变成向量。
2. **Positional Encoding**: 加上位置信息。
3. **Layers Stack**: 使用 `nn.ModuleList` 堆叠 $N$ 个 `EncoderLayer`。

In [ ]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.append(project_root)

import copy
import math
import torch
import torch.nn as nn
from src.models.transformer_components import PositionalEncoding, EncoderLayer 

def get_clones(module, N):
    """
    一个辅助函数：克隆 N 个相同的层
    """
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, d_ff, N=6, dropout=0.1):
        """
        Args:
            vocab_size: 词表大小 (比如 30000)
            d_model: 向量维度 (512)
            n_heads: 多头数 (8)
            d_ff: FFN隐藏层维度 (2048)
            N: 堆叠层数 (通常是 6 或 12)
        """
        super(TransformerEncoder, self).__init__()
        
        self.d_model = d_model
        self.N = N
        
        # 1. 词嵌入层 (Word Embedding)
        self.embedding = nn.Embedding(vocab_size, d_model)
        
        # 2. 位置编码 (Positional Encoding)
        self.pe = PositionalEncoding(d_model)
        
        # 3. 堆叠 N 层 EncoderLayer
        # 先实例化一个标准层
        base_layer = EncoderLayer(d_model, n_heads, d_ff, dropout)
        # 然后克隆 N 份
        self.layers = get_clones(base_layer, N)
        
        # 4. 最终的规范化层 (可选，但推荐)
        self.norm = nn.LayerNorm(d_model)
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, mask=None):
        """
        src: [Batch, Seq_Len] (注意：这里输入的是整数索引，不是向量了)
        mask: [Batch, 1, 1, Seq_Len]
        """
        # 1. Embedding
        # [Batch, Seq] -> [Batch, Seq, Dim]
        x = self.embedding(src)
        
        # 2. Scale Embedding (这是一个 trick)
        # 论文中提到 embedding 需要乘以 sqrt(d_model)，让数值变大一点，
        # 以便和 Positional Encoding (在 -1 到 1 之间) 相加时，
        # 原始语义信息不会被位置信息淹没。
        x = x * math.sqrt(self.d_model)
        
        # 3. Add Position Encoding
        x = self.pe(x)
        x = self.dropout(x)
        
        # 4. Pass through N layers
        for layer in self.layers:
            x = layer(x, mask)
            
        # 5. Final Norm
        return self.norm(x)

## 3 联调测试

要把整数输入进去，看看能不能吐出完美的特征向量。

In [4]:
# 1. 配置参数 (参考 BERT-Base)
vocab_size = 1000 # 假设词表只有 1000 个词
d_model = 512
n_heads = 8
d_ff = 2048
N = 6           # 堆叠 6 层
batch_size = 32
seq_len = 10

# 2. 实例化模型
model = TransformerEncoder(vocab_size, d_model, n_heads, d_ff, N)

# 3. 构造虚拟输入
# 注意：输入必须是 Long (整数) 类型，范围在 [0, vocab_size) 之间
src = torch.randint(0, vocab_size, (batch_size, seq_len))

# 4. 前向传播
output = model(src)

# 5. 验证
print("Input shape (Indices):", src.shape) # [32, 10]
print("Output shape (Vectors):", output.shape) # [32, 10, 512]

assert output.shape == (batch_size, seq_len, d_model)
print("✅ Transformer Encoder 构建成功！")

# 看看这栋楼有多重 (参数量)
total_params = sum(p.numel() for p in model.parameters())
print(f"模型总参数量: {total_params / 1e6:.2f} Million (M)")

Input shape (Indices): torch.Size([32, 10])
Output shape (Vectors): torch.Size([32, 10, 512])
✅ Transformer Encoder 构建成功！
模型总参数量: 19.43 Million (M)


## 4 深度解析 & 面试八股

### Q1: 为什么用 `nn.ModuleList` 而不是普通的 Python `list`？(Python 面试高频)

  * **错误写法**: `self.layers = [EncoderLayer(...) for _ in range(N)]`
  * **后果**: 如果你用普通 list，PyTorch 的 `model.parameters()` **扫描不到**这些层里的权重。这意味着你训练时，这些层的参数**不会更新**！
  * **正确写法**: `nn.ModuleList` 是专门为 PyTorch 设计的列表，它会自动把里面的 Layer 注册到模型里，告诉优化器：“这也是我要训练的孩子。”

### Q2: Embedding 为什么要乘以 $\sqrt{d_{model}}$？

  * **直觉**:
      * Positional Encoding 的值在 $[-1, 1]$ 之间。
      * Embedding 初始化后的值往往很小（方差为 1）。
      * 如果直接相加，**位置信息**可能会太强，干扰了**语义信息**。
      * 乘以 $\sqrt{512} \approx 22.6$，相当于把语义信号放大，让它占据主导地位，位置信息作为辅助。

### Q3: 这就是 BERT 吗？

  * **是的！**
  * **BERT-Base** = 12 层 Encoder, $d_{model}=768$。
  * **BERT-Large** = 24 层 Encoder, $d_{model}=1024$。
  * 你刚刚手写的代码，就是 NLP 领域过去 5 年最强霸主的核心架构。

## 🚀 今日复盘与下一步

恭喜你！到今天为止，你已经**亲手造出了**：

1.  **位置编码**: 让模型有时间观念。
2.  **多头注意力**: 让模型有全局视野。
3.  **Encoder Layer**: 让模型有推理能力。
4.  **Encoder Stack**: 让模型有深度智慧。

**你现在的工程文件夹里，已经躺着一个微型的 BERT 模型了。**

### 📅 明天的计划 (S2W5D6)

明天我们稍微轻松一点，进行 **[支线任务] ViT (Vision Transformer)** 的学习。
你会发现：**把你今天写的代码，输入接口稍微改一下（把图片切片），它立刻就变成了特斯拉机器人的视觉大脑。**